In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### ВАЖНО: 
- На сайте https://spb.cian.ru/ выдаётся только 54 страницы максимум по 28 объявлений на каждой, 
т.е. суммарно 1512 ссылок при выставлении одного фильтра

# Cобираем ссылки на квартиры (по районам СПБ)

Можно было бы написать одну функцию, которая бы прошла по номерам всех районов в СПБ и по всем страницам от 1 до 54, НО:

- Во некоторых районах количество квартир < 200, а это значит что поисковая выдача на сайте даст около 7 страниц объявлений. При прохождении парсером в цикле по страницам от 1 до 54, с 8ой страницы (меняя число в формуле) Циан будет выдавать все последующие шаги цикла первую страницу объявлений. Таким образом получится огромное количество дубликатов, которые потом просто придётся удалить, т.е. будет выполняться бесполезная работа, отнимающая много времени.


- На основании вышесказанного, для уменьшения времени работы парсера, решил разбить районы СПБ по группам, где количество объявлений близко к 1512 (максимальной видимой выдаче ЦИАНА). Код получится более громоздкий, но время обработки будет значительно быстрее.

### Группы с номерами районов, где количество объявлений близко к 1512: 

##### 1). Адмиралтейский, Екатерингофский, Коломна, Семёновский, Сенной, Автово, Дачное, Княжево, Нарвский 

district[0]=725&district[1]=730&district[2]=735&district[3]=739&district[4]=741&district[5]=743&district[6]=747&district[7]=754&district[8]=759

##### 2). Красненькая речкa, Морские ворота, Ульянка, Измайловское

district[0]=711&district[1]=720&district[2]=732&district[3]=751

##### 3). Курортный(весь), Петродворцовый(весь), Васильевский

district[0]=137&district[1]=141&district[2]=712

##### 4). № 7, Остров Декабристов, Морской, Металлострой, Петро-Славянка, Понтонный, Сапёрный, Усть-Ижора

district[0]=718&district[1]=721&district[2]=740&district[3]=785&district[4]=793&district[5]=794&district[6]=795&district[7]=796

##### 5). Гавань

district[0]=728

##### 6).  Колпино, Новоизмайловское, Пулковский мередиан

district[0]=731&district[1]=749&district[2]=792

##### 7). Гагаринское

district[0]=767

##### 8). Звёздное

district[0]=761

##### 9). Московская застава, Озеро Долгое, Лисий Нос

district[0]=708&district[1]=737&district[2]=752

##### 10). № 65, Лахта-Ольгино, Александровская, Павловск

district[0]=715&district[1]=726&district[2]=757&district[3]=764

##### 11). Коломяги

district[0]=724

##### 12). Комендантский аэродром, Тярлево, Парнас, Сосновское, Левашово

district[0]=700&district[1]=717&district[2]=746&district[3]=760&district[4]=784

##### 13). Черная речка

district[0]=742

##### 14). Юнтолово

district[0]=727

##### 15). Пушкин, №15

district[0]=734&district[1]=771

##### 16). Шушары

district[0]=688

##### 17). Парголово

district[0]=723

##### 18). Сампсониевское, Шувалово-Озерки, № 21, Академическое

district[0]=744&district[1]=763&district[2]=773&district[3]=779

##### 19). Светлановское, Гражданка, Северный

district[0]=756&district[1]=772&district[2]=774

##### 20). Пискарёвка, Прометей, Финляндский, Большая Охта, Ржевка

district[0]=768&district[1]=769&district[2]=770&district[3]=778&district[4]=788

##### 21). Полюстрово

district[0]=780

##### 22). Малая Охта, Пороховые, Горелово, Константиновское, Красное Село, Кронштадт, Урицк

district[0]=691&district[1]=692&district[2]=704&district[3]=710&district[4]=713&district[5]=716&district[6]=789

##### 23). Южно-Приморский

district[0]=722

##### 24). Сосновая поляна, Юго-Запад, Аптекарский остров

district[0]=719&district[1]=729&district[2]=753

##### 25). Введенский, Кронверкское, Петровский

district[0]=736&district[1]=748&district[2]=750

##### 26). Посадский, Чкаловское, № 72, № 75

district[0]=693&district[1]=733&district[2]=758&district[3]=776

##### 27). Балканский, Волковское, Георгиевский, Купчино, № 78, Лиговка-Ямская

district[0]=687&district[1]=689&district[2]=690&district[3]=762&district[4]=765&district[5]=775

##### 28). Владимирский, Дворцовый, Смольнинское

district[0]=707&district[1]=738&district[2]=755

##### 29). Литейный, Ивановский, Невская застава, Правобережный, Оккервиль

district[0]=745&district[1]=766&district[2]=777&district[3]=783&district[4]=791

##### 30). №54

district[0]=786

##### 31). Рыбацкое, Народный

district[0]=787&district[1]=790

##### 32).Невский, Обуховский

district[0]=781&district[1]=782

In [28]:
#Скомпануем всё в один список:
spb_areas=['district[0]=725&district[1]=730&district[2]=735&district[3]=739&district[4]=741&district[5]=743&district[6]=747&district[7]=754&district[8]=759',
          'district[0]=711&district[1]=720&district[2]=732&district[3]=751',
          'district[0]=137&district[1]=141&district[2]=712',
          'district[0]=718&district[1]=721&district[2]=740&district[3]=785&district[4]=793&district[5]=794&district[6]=795&district[7]=796',
          'district[0]=728',
          'district[0]=731&district[1]=749&district[2]=792',
          'district[0]=767',
          'district[0]=761',
          'district[0]=708&district[1]=737&district[2]=752',
          'district[0]=715&district[1]=726&district[2]=757&district[3]=764',
          'district[0]=724',
          'district[0]=700&district[1]=717&district[2]=746&district[3]=760&district[4]=784',
          'district[0]=742',
          'district[0]=727',
          'district[0]=734&district[1]=771',
          'district[0]=688',
          'district[0]=723',
          'district[0]=744&district[1]=763&district[2]=773&district[3]=779',
          'district[0]=756&district[1]=772&district[2]=774',
          'district[0]=768&district[1]=769&district[2]=770&district[3]=778&district[4]=788',
          'district[0]=780',
          'district[0]=691&district[1]=692&district[2]=704&district[3]=710&district[4]=713&district[5]=716&district[6]=789',
          'district[0]=722',
          'district[0]=719&district[1]=729&district[2]=753',
          'district[0]=736&district[1]=748&district[2]=750',
          'district[0]=693&district[1]=733&district[2]=758&district[3]=776',
          'district[0]=687&district[1]=689&district[2]=690&district[3]=762&district[4]=765&district[5]=775',
          'district[0]=707&district[1]=738&district[2]=755',
          'district[0]=745&district[1]=766&district[2]=777&district[3]=783&district[4]=791',
          'district[0]=786',
          'district[0]=787&district[1]=790',
          'district[0]=781&district[1]=782']

In [30]:
#Функция парсинга ссылок с сайта:
def hrefFinder():
    href=[]                  
    for area in spb_areas:  
        for i in range(1,55):
            page = 'https://spb.cian.ru/cat.php?deal_type=sale&'+area+'&engine_version=2&offer_type=flat&p='+str(i)+'&region=2'
            response = requests.get(page)
            html = response.content
            soup = BeautifulSoup(html,'html.parser')
            dirty_href = soup.findAll('a', attrs = {'class':"_93444fe79c--link--39cNw"})
            for item in dirty_href:
                href.append(item.attrs['href'])
    return(href)

In [31]:
href = hrefFinder()

In [32]:
#Преобразуем в DataFrame:
href_df = pd.DataFrame(href)

In [36]:
#Удалим дубликаты:
href_df = href_df.drop_duplicates()

In [37]:
#Обновим индексы:
href_df.reset_index(inplace=True)
href_df = href_df.drop('index',axis=1)

In [40]:
href_df.to_csv('hrefs_from_cian.csv', index=False, header=False)